In [2]:
import pandas as pd

filmi = pd.read_csv('../../02-zajem-podatkov/predavanja/obdelani-podatki/filmi.csv', index_col='id')
osebe = pd.read_csv('../../02-zajem-podatkov/predavanja/obdelani-podatki/osebe.csv', index_col='id')
vloge = pd.read_csv('../../02-zajem-podatkov/predavanja/obdelani-podatki/vloge.csv')
zanri = pd.read_csv('../../02-zajem-podatkov/predavanja/obdelani-podatki/zanri.csv')


In [ ]:
pojavitve_zanrov = zanri.groupby("zanr").size()
verjetnosti_zanrov = pojavitve_zanrov / len(filmi)
pojavitve_zanrov

zanr
Action        2506
Adventure     1710
Animation      470
Biography      671
Comedy        3662
Crime         2061
Drama         5680
Family         470
Fantasy        745
Film-Noir       58
History        341
Horror        1267
Music          276
Musical        135
Mystery       1038
Reality-TV       1
Romance       1732
Sci-Fi         726
Sport          201
Thriller      1712
War            226
Western        118
dtype: int64

In [5]:
beseda = "To je ena beseda"

In [16]:
odstrani_konce("ringings", ["ing", "d", "s"])

'ringing'

In [40]:
def odstrani_konce(beseda, konci):
    for konec in konci:
        if beseda.endswith(konec):
            return beseda[:-len(konec)]
    return beseda
    
def koren_besede(beseda):
    beseda = ''.join(znak for znak in beseda if znak.isalpha())
    if not beseda:
        return '$'
    beseda = odstrani_konce(beseda, ["ing", "ed", "en", "s"])
    beseda = beseda.rstrip('aeiou')
    if not beseda:
        return '@'
    return beseda

def koreni_besed(niz):
    return pd.Series(sorted({
        koren_besede(beseda) for beseda in niz.replace('-', ' ').lower().split() if beseda
    }))

In [41]:
opis = """In 1938, after his father Professor Henry Jones, Sr.
          goes missing while pursuing the Holy Grail, Professor
          Henry "Indiana" Jones, Jr. finds himself up against
          Adolf Hitler's Nazis again to stop them from obtaining
          its powers."""
koreni_besed(opis)

0             $
1         adolf
2         after
3         again
4       against
5        father
6          find
7          from
8             g
9         grail
10            h
11        henry
12      himself
13       hitler
14         holy
15           in
16       indian
17           it
18          jon
19           jr
20         miss
21          naz
22       obtain
23        power
24    professor
25         purs
26           sr
27         stop
28            t
29           th
30         them
31           up
32         whil
dtype: object

In [50]:
razpredelnica_s_koreni_filmov = filmi.opis.apply(koreni_besed)

In [58]:
koreni_filmov = (
    razpredelnica_s_koreni_filmov.unstack(  # vnose v razpredelnici razpotegnemo v eno vrsto (z gnezdenim indeksom)
    ).reset_index(                          # podatke iz indeksa pretvorimo v razpredelnico
    )[
        ['id', 0]                           # izberemo le relevantna dva stolpca
    ].rename(
        {"id": "film", 0: "koren"},         # stolpcema damo smiselni imeni 
        axis=1                              # s tem povemo, da preimenujemo stolpce in ne vrstic
    ).dropna(                               # odstranimo manjkajoče vnose
    )
)
koreni_filmov

,film,koren
0,4972,affect
1,6864,@
2,9968,@
3,10323,@
4,12349,abandon
...,...,...
525560,862856,with
529018,6433880,with
535559,862856,woman
539017,6433880,world


In [64]:
koreni_zanrov = pd.merge(koreni_filmov, zanri)
koreni_zanrov

,film,koren,zanr
0,4972,affect,Drama
1,4972,affect,History
2,4972,affect,War
3,4972,and,Drama
4,4972,and,History
...,...,...,...
590656,21279138,of,Drama
590657,21279138,story,Drama
590658,21279138,tell,Drama
590659,21279138,th,Drama


In [ ]:
tragikomedije = pd.merge(
        koreni_zanrov[(koreni_zanrov.zanr == "Comedy") & (koreni_zanrov.koren == "tragedy")],
        filmi, 
        left_on="film",
        right_on="id"
    )
tragikomedije

,film,koren,zanr,naslov,dolzina,leto,ocena,metascore,glasovi,zasluzek,oznaka,opis
0,300051,tragedy,Comedy,Jersey Girl,102,2004,6.2,43.0,66702,25266129.0,PG-13,A widower must look after his precocious daugh...
1,332136,tragedy,Comedy,If Only,92,2004,7.0,NaN,32719,NaN,PG-13,It takes a tragedy to teach the young business...
2,378947,tragedy,Comedy,Melinda and Melinda,99,2004,6.4,54.0,32447,3825351.0,PG-13,"Two alternating stories, one comedy and the ot..."
3,485510,tragedy,Comedy,Killing the Shadows,135,2006,7.5,NaN,12537,NaN,NaN,A story about the tragedy and the danger of hu...
4,1247692,tragedy,Comedy,Shrink,104,2009,6.6,40.0,18889,189332.0,R,"Unable to cope with a recent personal tragedy,..."


In [87]:
stolpec_verjetnosti_korenov_pri_zanrih = koreni_zanrov.groupby(["zanr", "koren"]).size() / pojavitve_zanrov
verjetnosti_korenov_pri_zanrih = stolpec_verjetnosti_korenov_pri_zanrih.unstack().transpose().fillna(1 / len(filmi))
verjetnosti_korenov_pri_zanrih

zanr,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,Film-Noir,...,Music,Musical,Mystery,Reality-TV,Romance,Sci-Fi,Sport,Thriller,War,Western
koren,,,,,,,,,,,,,,,,,,,,,
$,0.092977,0.084211,0.057447,0.236960,0.089022,0.089277,0.119014,0.080851,0.075168,0.034483,...,0.094203,0.074074,0.085742,1.0000,0.092956,0.117080,0.144279,0.090537,0.203540,0.118644
@,0.906225,0.883041,0.876596,0.785395,0.883943,0.915575,0.895070,0.851064,0.897987,0.948276,...,0.873188,0.866667,0.946050,1.0000,0.886836,0.921488,0.845771,0.941005,0.902655,0.864407
aang,0.000399,0.000585,0.000100,0.000100,0.000100,0.000100,0.000100,0.002128,0.000100,0.000100,...,0.000100,0.000100,0.000100,0.0001,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100
aaron,0.000399,0.000585,0.000100,0.000100,0.000546,0.000485,0.000176,0.000100,0.000100,0.000100,...,0.000100,0.000100,0.000963,0.0001,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100
aart,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000176,0.000100,0.000100,0.000100,...,0.000100,0.000100,0.000100,0.0001,0.000577,0.000100,0.000100,0.000100,0.000100,0.000100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
æon,0.000399,0.000585,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,...,0.000100,0.000100,0.000100,0.0001,0.000100,0.001377,0.000100,0.000100,0.000100,0.000100
çanakkal,0.000100,0.000100,0.000100,0.000100,0.000273,0.000100,0.000100,0.000100,0.000100,0.000100,...,0.000100,0.000100,0.000100,0.0001,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100
édith,0.000100,0.000100,0.000100,0.001490,0.000100,0.000100,0.000176,0.000100,0.000100,0.000100,...,0.003623,0.000100,0.000100,0.0001,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100


zanr,koren,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,...,Music,Musical,Mystery,Reality-TV,Romance,Sci-Fi,Sport,Thriller,War,Western
0,$,0.092977,0.084211,0.057447,0.23696,0.089022,0.089277,0.119014,0.080851,0.075168,...,0.094203,0.074074,0.085742,1.0000,0.092956,0.11708,0.144279,0.090537,0.20354,0.118644
4,aart,0.000100,0.000100,0.000100,0.00010,0.000100,0.000100,0.000176,0.000100,0.000100,...,0.000100,0.000100,0.000100,0.0001,0.000577,0.00010,0.000100,0.000100,0.00010,0.000100


In [103]:
def zanri(opis):
    koreni_filma = koreni_besed(opis)
    r = verjetnosti_korenov_pri_zanrih.reset_index()
    verjetnosti = r[r.koren.isin(koreni_filma)].product(numeric_only=True) * verjetnosti_zanrov
    return verjetnosti.sort_values(ascending=False).head(5)

zanri("A cruise for the super-rich sinks thus leaving survivors, including a fashion model celebrity couple, trapped on an island.")

zanr
Thriller     4.803878e-34
Mystery      4.045567e-34
Adventure    2.189918e-34
Comedy       8.256366e-35
Drama        8.241234e-35
dtype: float64